In [17]:
import pandas as pd
import numpy as np
import json

In [18]:
def pprint(obj):
    print(json.dumps(obj, indent=2))


params = json.loads(open('algoparams_from_ui.json').read())

print("\nTarget:")
target = params['design_state_data']['target']
pprint(target)

print("\nFeature Handling:")
feature_handling = params['design_state_data']['feature_handling']
pprint(feature_handling)

dataset_name = params['design_state_data']['session_info']['dataset']
print(f"\nDataset name: {dataset_name}")



Target:
{
  "prediction_type": "Regression",
  "target": "petal_width",
  "type": "regression",
  "partitioning": true
}

Feature Handling:
{
  "sepal_length": {
    "feature_name": "sepal_length",
    "is_selected": true,
    "feature_variable_type": "numerical",
    "feature_details": {
      "numerical_handling": "Keep as regular numerical feature",
      "rescaling": "No rescaling",
      "make_derived_feats": false,
      "missing_values": "Impute",
      "impute_with": "Average of values",
      "impute_value": 0
    }
  },
  "sepal_width": {
    "feature_name": "sepal_width",
    "is_selected": true,
    "feature_variable_type": "numerical",
    "feature_details": {
      "numerical_handling": "Keep as regular numerical feature",
      "rescaling": "No rescaling",
      "make_derived_feats": false,
      "missing_values": "Impute",
      "impute_with": "custom",
      "impute_value": -1
    }
  },
  "petal_length": {
    "feature_name": "petal_length",
    "is_selected": true,


In [19]:
df = pd.read_csv(r"iris.csv")

In [20]:
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [21]:
df.isnull()

,sepal_length,sepal_width,petal_length,petal_width,species
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
145,False,False,False,False,False
146,False,False,False,False,False
147,False,False,False,False,False
148,False,False,False,False,False


In [22]:
df.isnull().sum()

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
species         0
dtype: int64

# Data Preprocessing

In [23]:
for col, feature in feature_handling.items():
    if not feature['is_selected']:
        df.drop(col, axis=1, inplace=True)

    if feature["feature_variable_type"] == "numerical":

        if feature['feature_details']["missing_values"] == "Impute":
            if feature['feature_details']['impute_with'] == "Average of values":
                df[col].fillna(df[col].mean(), inplace=True)
            if feature['feature_details']['impute_with'] == "custom":
                df[col].fillna(feature['feature_details']
                               ['impute_value'], inplace=True)
            else:
                AssertionError(
                    f"Unknown imputation method: {feature['feature_details']['impute_with']}")
    elif feature["feature_variable_type"] == "text":
        labels = {key: num for num, key in enumerate(df[col].unique())}
        df[col] = df[col].apply(lambda x: labels[x])

    else:
        AssertionError(
            f"Unknown feature type: {feature['feature_variable_type']}")

 # Feature Reduction

In [24]:
config = params['design_state_data']['feature_reduction']

target_col = target['target']

X = df.drop(target_col, axis=1).values
y = df[target_col].values

if config['feature_reduction_method'] == "Tree-based":
    if target['type'] == "regression":
        from sklearn.ensemble import RandomForestRegressor
        from sklearn.feature_selection import SelectFromModel
        sel = SelectFromModel(RandomForestRegressor(n_estimators=int(
            config['num_of_trees']), max_depth=int(config['depth_of_trees'])))

    elif target['type'] == "classification":
        from sklearn.ensemble import RandomForestClassifier
        from sklearn.feature_selection import SelectFromModel
        sel = SelectFromModel(RandomForestClassifier(n_estimators=int(
            config['num_of_trees']), max_depth=int(config['depth_of_trees'])))

    sel.fit(X, y)
    feature_importance = sel.estimator_.feature_importances_
    sorted_indices = np.argsort(feature_importance)[::-1]
    keep_columns = df.columns[np.concatenate((sorted_indices[:int(
        config['num_of_features_to_keep'])], [list(df.columns).index(target_col)]))]
    df = df[keep_columns]

elif config['feature_reduction_method'] == "No Reduction":
    pass

elif config['feature_reduction_method'] == "Correlation with target":
    corr = df.corr()[target_col].drop(target_col)
    sorted_cor = sorted(dict(abs(corr).items()).items(), key=lambda x: x[1], reverse=True)[
        :int(config['num_of_features_to_keep'])]
    keep_columns = np.array([key for key, value in sorted_cor] + [target_col])
    df = df[keep_columns]

elif config['feature_reduction_method'] == "Principal Component Analysis":
    from sklearn.decomposition import PCA

    pca = PCA(n_components=int(config['num_of_features_to_keep']))
    pca.fit(X)
    print(f"Explained variance: {pca.explained_variance_ratio_}")
    X = pca.transform(X)

else:
    AssertionError(
        f"Unknown feature reduction method: {config['feature_reduction_method']}")

algorithms = params['design_state_data']['algorithms']


In [29]:
def sklearn_model(algo_name, hyperparameters):
    model = None
    model_name = hyperparameters.pop('model_name')

    if model_name == "Random Forest Classifier":
        from sklearn.ensemble import RandomForestClassifier
        model = RandomForestClassifier(**hyperparameters)

    elif model_name == "Random Forest Regressor":
        from sklearn.ensemble import RandomForestRegressor

        from sklearn.model_selection import GridSearchCV
        parameters = {
            'n_estimators': [hyperparameters["min_trees"], hyperparameters["max_trees"]],
            'max_depth': [hyperparameters["min_depth"], hyperparameters["max_depth"]],
            'min_samples_leaf': [hyperparameters["min_samples_per_leaf_min_value"],
                                 hyperparameters["min_samples_per_leaf_max_value"]]
            }
        model = GridSearchCV(RandomForestRegressor(), parameters, cv=5, n_jobs=-1)
        model.fit(X, y)
        print(f"Best parameters: {model.best_params_}")
        print(f"Best score: {model.best_score_}")
        return model.best_estimator_
        # cam ne done similiarly for other models
    elif model_name == "Gradient Boosted Trees":
        pass
    elif model_name == "Gradient Boosted Trees":
        pass
    elif model_name == "LinearRegression":
        pass
    elif model_name == "LogisticRegression":
        pass
    elif model_name == "RidgeRegression":
        pass
    elif model_name == "Lasso Regression":
        pass
    elif model_name == "Lasso Regression":
        pass
    elif model_name == "XG Boost":
        pass
    elif model_name == "Decision Tree":
        pass
    elif model_name == "Decision Tree":
        pass
    elif model_name == "Support Vector Machine":
        pass
    elif model_name == "Stochastic Gradient Descent":
        pass
    elif model_name == "KNN":
        pass
    elif model_name == "Extra Random Trees":
        pass
    elif model_name == "Neural Network":
        pass
    
    return model


print()
print()
print()



In [30]:
for algo_name, hyperparameters in algorithms.items():
    is_selected = hyperparameters.pop('is_selected')
    name = hyperparameters['model_name']


    if not is_selected:
        continue

    model = sklearn_model(algo_name, hyperparameters)
    print(name)
    pprint(hyperparameters)
    
    if not model is None:
        break

print("Finished")

Best parameters: {'max_depth': 25, 'min_samples_leaf': 5, 'n_estimators': 10}
Best score: 0.44866412159074975
Random Forest Regressor
{
  "min_trees": 10,
  "max_trees": 20,
  "feature_sampling_statergy": "Default",
  "min_depth": 20,
  "max_depth": 25,
  "min_samples_per_leaf_min_value": 5,
  "min_samples_per_leaf_max_value": 10,
  "parallelism": 0
}
Finished
